# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Открываем файл с данными для изучения общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Вывод по таблице:
1) метод info() показывает, что есть пропуски в столбцах days_employed и total_income (возможно, в строках с одним индексом). В days_employed в основном отрицательные значения, возможно это ошибка, так как есть и положительные. В итоговых задачах нет вопроса о зависимости выплаты кредита от стажа работы, поэтому колонку решено удалить. В столбце total_income можно заменить пропуски средними арифметическими значениями, так как удаление их исказит расчёты.  

2) с типами данных нет особых наблюдений, хотя тип float не так уж необходим, пожалуй,стоит привести их к целому значению.  

3) в столбце education значения нужно привести к нижнему регистру.

## Предобработка данных

### Обработка пропусков

In [4]:
#df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)].info()
# строки те же самые

del df['days_employed']# удален столбец 
df.columns # проверка столбцов таблицы

df[df['total_income'].isnull()].count()# количество пропущенных значений NaN 
income_type_transform = df.groupby('income_type')['total_income'].transform('mean')
income_type_mean = df.groupby('income_type')['total_income'].mean()
df['total_income'].fillna(income_type_transform, inplace=True)#замена NaN в столбце средними значениями 
display(df.isna().sum())# проверка количества пропусков

#df['gender'].value_counts()
df = df.drop(df[df['gender']=='XNA'].index) #удаление в столбце значение XNA
display(df['gender'].value_counts())

#df['dob_years'].mean() # замена возраста 0 на среднее значение по столбцу
df['dob_years'] = df['dob_years'].replace(0, 43.29337979094077)
#df['dob_years'][df['dob_years'] == 0].count()# проверка результата
#df['dob_years'].min()


children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

F    14236
M     7288
Name: gender, dtype: int64

**Вывод**

Пропуски в столбце total_income заполнены средними значениями методом fillna.Обнаружено значение XNA в столбце gender и удалено методом drop. В столбце age нулевой возраст замещен на средний методом replace.

### Замена типа данных

In [5]:
df['total_income'] = df['total_income'].astype('int64')
df['dob_years'] = df['dob_years'].astype('int64')#замена float на int
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   dob_years         21524 non-null  int64 
 2   education         21524 non-null  object
 3   education_id      21524 non-null  int64 
 4   family_status     21524 non-null  object
 5   family_status_id  21524 non-null  int64 
 6   gender            21524 non-null  object
 7   income_type       21524 non-null  object
 8   debt              21524 non-null  int64 
 9   total_income      21524 non-null  int64 
 10  purpose           21524 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


None

**Вывод**

Замещаем данные float в столбце total_income на int методом astype, чтобы было удобнее воспринимать данные в таблице. 

### Обработка дубликатов

In [6]:
#df['education'].value_counts() # количество дубликатов значений в столбце 
df['education'] = df['education'].str.lower()# приводим значения в столбце к нижнему регистру
df['education'].value_counts() # значения в столбце не повторяются

display(df.duplicated().sum())# количество дубликатов 71
df = df.drop_duplicates().reset_index(drop = True)# удаление дубликатов
display(df.duplicated().sum())# 0 -теперь дубликаты убраны

71

0

**Вывод**

Необходимо обратить внимание на заполнения ячеек с уровнем образования клиентов. Ошибки "среднее", "Среднее" или "СРЕДНЕЕ" могли возникнуть из-за возможного заполнения разными людьми вручную. Для поиска дубликатов используется метод value_counts, sum, для удаления - drop_duplicates.

### Лемматизация

In [7]:
m = Mystem()

def lemma_column(row):
    lemmas = m.lemmatize(row)                     
    return lemmas

df['purpose_lem'] = df['purpose'].apply(lemma_column)
#display(df.head(20))
df['purpose_lem'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод**

Создан отдельный столбец 'purpose_lem' для удобства чтения информации. Из общего количества можно сделать вывод, что есть 4 основные группы целей для оформления кредита: свадьба, покупка недвижимости, покупка автомобиля, образование.

### Категоризация данных

In [8]:
# функция для категорий по целям кредита
def replace_target(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'автомобиль' in row:
        return 'авто'
    if 'недвижимость' in row or 'жилье' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'
    
    
#отдельный столбец, для удобства дальнейшего анализа   
df['purpose_id'] = df['purpose_lem'].apply(replace_target)
df['purpose_id'].value_counts()


display(pd.qcut(df['total_income'], q=4))#разбиваем уровень дохода по квантилям

def replace_income(total_income):
    if 20666 <= total_income <= 107620:
        return 'низкий'
    if 107620 <= total_income <= 151876:
        return 'средний'
    if 151876 <= total_income <= 202417:
        return 'выше среднего'
    if 202417 <= total_income <= 2265604:
        return 'высокий'
    if total_income > 2265604:
        return 'очень высокий'
    
     
df['total_income_id'] = df['total_income'].apply(replace_income)

# функция для категоризации по детям: 1 — дети есть, 0 — детей нет.
def determine_children(children):
    if children > 0: 
        return 1
    else: 
        return 0


display(df)

0        (202417.0, 2265604.0]
1         (107620.0, 151876.0]
2         (107620.0, 151876.0]
3        (202417.0, 2265604.0]
4         (151876.0, 202417.0]
                 ...          
21448    (202417.0, 2265604.0]
21449     (151876.0, 202417.0]
21450    (20666.999, 107620.0]
21451    (202417.0, 2265604.0]
21452    (20666.999, 107620.0]
Name: total_income, Length: 21453, dtype: category
Categories (4, interval[float64]): [(20666.999, 107620.0] < (107620.0, 151876.0] < (151876.0, 202417.0] < (202417.0, 2265604.0]]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,purpose_id,total_income_id
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",авто,средний
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,выше среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,высокий
21449,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",авто,выше среднего
21450,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,низкий
21451,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",авто,высокий


**Вывод**

Категоризация сделана по целям кредита, уровню дохода и наличию детей, для последующей формулировки ответов на поставленные вопросы.

##  Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
#df['children'].unique() # посмотрим, какое количество детей представлено в строках
df['children'] = df['children'].replace(-1,1) # заменяем явно некорректные значения в столбце: -1 на 1
df['children'] = df['children'].replace(20,2) # заменяем возможную ошибку 20 на 2
display(df['children'].value_counts())# проверяем, теперь данные исправлены

df['child_sub'] = df['children'].apply(determine_children)

ch = df.pivot_table(index = ['child_sub'], values = 'debt')
ch.head(10)

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

,debt
child_sub,
0,0.075444
1,0.092082


**Вывод**

Небольшой, но перевес обязательности в пользу клиентов без детей. Причиной, возможно, могут быть внезапные расходы, связанные с ними.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
fs = df.pivot_table(index = ['family_status'], values='debt', aggfunc = 'count')
sum_fs = df.pivot_table(index = ['family_status'], values='debt', aggfunc = 'sum')
debt_fs = sum_fs / fs
debt_fs.sort_values(by ='debt', ascending = False)
#print(df['family_status'].value_counts())

,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093494
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


**Вывод**

Исходя из данных, люди, бывшие и находящиеся в официальном браке, самые обязательные. Тут может быть несколько причин. Возможно люди, состоящие в брачных отношениях (или бывшие в них)более ответственны. Возможно у них другие цели кредита, скорее всего чаще их цель - ипотека, по которой, как мы видим из данных этой же таблице более низкие долговые показатели. У тех, кто пока не состоял в отношениях, цели скорее всего личные, а они, с точки зрения долга, менее возвратные.

##  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
inc = df.pivot_table(index = 'total_income_id', values='debt', aggfunc = 'count')
inc_sum = df.pivot_table(index = 'total_income_id', values='debt', aggfunc = 'sum')
total_income_debt = inc_sum / inc
total_income_debt.sort_values(by ='debt', ascending = False)

,debt
total_income_id,
средний,0.088570
выше среднего,0.084909
низкий,0.079605
высокий,0.070293


**Вывод**

Как ни странно, люди с низким доходом наиболее обязательны, по сравнению с людьми у кого доход высокий или выше среднего. Для людей с низким доходом вероятно важна хорошая кредитная история и, кроме того, с низким доходом сложнее выплачивать пенни за просрочку по процентам, а люди с очень высоким доходом не имеют причин для задержек.

## Как разные цели кредита влияют на его возврат в срок?

In [12]:
purpose_group = df.groupby('purpose_id').agg({'debt':['sum','count']})
conversion = purpose_group['debt']['sum'] / purpose_group['debt']['count']
display(conversion)

purpose_id
авто            0.093590
недвижимость    0.072340
образование     0.092200
свадьба         0.080034
dtype: float64

**Вывод**

Больше всего просрочек у кредитов на автомобиль и образование, их доля 0.092 и 0.093.
Кредит на личные цели и кредит на недвижимось имеют меньше просрочек. Возможно, в покупке автомобиля больше спонтанности, поэтому заемщики могут неверно оценить свои возможности.Человек, выбирая ипотеку, лучше оценивает перспективу, так как срок платежа очень большой. Помимио прочего, недвижимость можно продать на определенном этапе не теряя в цене и компенсируя проценты, в отличие от автомобиля, который падает в цене или приобретения образовательных услуг, которые не возмещаются.

## Общий вывод

Из полученных данных можно сделать вывод, что самыми надежными заемщиками являются люди, которые берут ипотеку, с низким или очень высоким уровнем заработка, без детей, состоящие или бывшие в официальном браке.